In [17]:
import pandas as pd
import nltk

In [5]:
cosmetic = pd.read_csv('../dataset/cosmetics_reviews_final.csv')
reviews = cosmetic['review']
reviews[:10]

0    트리플어피치 쓰는데 음    발색도 별로 텍스쳐가 정말 뭐라하지 너무 기름진데 촉촉...
1                              치크로만 쓰는데 굉장히 여리여리라게 발색됨
2    색은 둘째치고 틴트가 지속력이 없음  올려도 지워지니 살 필요가 없다  립브러쉬 쓰...
3    루즈G 립스틱이랑 세트인 컴팩트   일단 컴팩트 디자인이 엄청 다양해요  심플한 검...
4    164 쓰는데 고급진 핑크에요   발림성이 너무 맘에 들었어요 지속은 좀 아쉬웠고 ...
5    원래 립스틱타입 쓰다가 매장언니 추천으로 샀는데 이가격에도 쓰게되는 애증의 아이템임...
6    여름이라 팔꿈치랑 복숭아뼈 같은부위는 되게 자주 스크럽하는 편인데 꽤 깨끗하게 잘 ...
7       케이스 이뻐서 샀는데 너무 촉촉함 지속력이 너무 없다    그래도 꿋꿋이 발랐었어요
8    상큼한 레몬향 촉촉하고 산뜻하게 보습이 돼요 겨울에는 좀 부족할 듯 다른 립밤들보다...
9      케이스 귀엽  향은 진짜 좋음  이것만 바르면 목이 따갑고 먼지 먹은 느낌 정말 싫음
Name: review, dtype: object

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(reviews, test_size=.2, random_state =0)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import words

vectorizer = CountVectorizer(analyzer = 'word', 
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다. 
                             max_features = 90000
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=90000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None,
        vocabulary={'unremounted', 'lathee', 'Loliginidae', 'Cliona', 'pincher', 'coexpire', 'amyotrophic', 'laddery', 'Australianism', 'outbark', 'amyelencephalic', 'translucently', 'manyberry', 'unbragged', 'pharmacography', 'Heptameron', 'dozer', 'fracedinous', 'counterscarp', 'stomatiferous', 'speck', '...l', 'reaudit', 'mouthroot', 'unimpregnate', 'beringleted', 'baseness', 'zygantra', 'preposterously'})

In [11]:
pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer(smooth_idf = False)),
])  
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=90000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        stri...('tfidf', TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True))])

In [12]:
%time X_train_tfidf_vector = pipeline.fit_transform(X_train)

CPU times: user 1.46 s, sys: 37.7 ms, total: 1.49 s
Wall time: 1.52 s


/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(float(n_samples) / df) + 1.0


In [14]:
%time X_test_tfidf_vector = pipeline.fit_transform(X_test)

CPU times: user 637 ms, sys: 24.6 ms, total: 662 ms
Wall time: 659 ms


/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(float(n_samples) / df) + 1.0


In [15]:
X_train_tfidf_vector

<16977x235892 sparse matrix of type '<class 'numpy.float64'>'
	with 135 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)